# EXPLORATORY DATA ANALYSIS

#### IMPORT DES DONNEES

In [1]:
import pandas as pd
df = pd.read_csv("../SBAnational.csv")
data = df.copy()
data = data.drop(['ChgOffDate', 'DisbursementDate', 'DisbursementGross','BalanceGross', 'ChgOffPrinGr'], axis=1)
data.head(3)

C:\Users\Thibaut\AppData\Local\Temp\ipykernel_24888\3806634080.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../SBAnational.csv")


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,2.0,0,0,1,0,N,Y,P I F,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,2.0,0,0,1,0,N,Y,P I F,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,1.0,0,0,1,0,N,N,P I F,"$287,000.00","$215,250.00"


### SWEETVIZ

In [2]:
# import sweetviz as sv
# df_test['ApprovalFY'] = pd.to_numeric(df_test['ApprovalFY'], errors='coerce')
# my_report = sv.analyze(df_test)
# my_report.show_html() # Default arguments will generate to "SWEETVIZ_REPORT.html"

### YDATA

In [3]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(df, title="Profiling Report")
# profile

### NETTOYAGE DES DONNEES

In [4]:
data.dtypes 

LoanNr_ChkDgt      int64
Name              object
City              object
State             object
Zip                int64
Bank              object
BankState         object
NAICS              int64
ApprovalDate      object
ApprovalFY        object
Term               int64
NoEmp              int64
NewExist         float64
CreateJob          int64
RetainedJob        int64
FranchiseCode      int64
UrbanRural         int64
RevLineCr         object
LowDoc            object
MIS_Status        object
GrAppv            object
SBA_Appv          object
dtype: object

In [5]:
def amount_to_float(s: str) -> float:
    """Converts a 'string' amount in $ to its float value"""
    return float(s[1:].replace(',', ''))

data["GrAppv"] = data["GrAppv"].apply(amount_to_float)
data["SBA_Appv"] = data["SBA_Appv"].apply(amount_to_float)
data 

TypeError: 'int' object is not iterable

In [ ]:
# data.dtypes 

In [ ]:
# def str_to_int(s: str) -> int:
#     """Converts a 'string' digit into an integer and deletes string characters if necessary"""
#     for char in s:
#         if char.isdigit() == False:
#             s.remove(char)
#         else:
#             pass
#     return int(s)

# data["ApprovalFY"] = data["ApprovalFY"].apply(str_to_int)
# data 

# MACHINE LEARNING

### PYCARET

In [ ]:
# from pycaret.classification import *
# df_test = data.copy()
# df_test = df_test.dropna()
# session = setup(data=df_test, target='MIS_Status', normalize=True, train_size=0.8, data_split_stratify=True, fold=5, session_id=0)

In [ ]:
# top_models = compare_models(sort="Accuracy")
# print(top_models)